In [ ]:
# default_exp hpar

# H Parameter
> A reader for the binary H parameter map

In [ ]:
# export
from pathlib import Path

import dask.array as da
import dask
import hvplot.xarray
import numpy as np
import xarray as xr

basepath = Path("/luna4/maye/h_parameter_maps")
lons_path = basepath / "hpar_global128ppd_v1c.lon.bin"
lats_path = basepath / "hpar_global128ppd_v1c.lat.bin"
hpar_path = basepath / "hpar_global128ppd_v1c.bin"


def memmap_binary(fname):
    # one extra element for both lats (the 0-line) and lons (360==0)
    cols = 128 * 360 + 1
    rows = 128 * 140 + 1
    # this doesn't read, just maps the diskfile into a virtual array
    return np.memmap(fname, np.float32, mode="r", shape=(rows, cols), order="F")


def read_hpar_binary(fname=hpar_path):
    """Reading Paul Hayne's binary global 128 ppd H-parameter map.

    Parameters
    ----------
    fname: str, pathlib.Path
        Path to binary 128 ppd map file binary (ending in .bin)
    """
    if Path(fname).suffix != ".bin":
        print("not an expected filename ending in .bin .")
        return

    arr = memmap_binary(hpar_path)

    # create a dask array
    a = da.from_array(arr)

    # replacing the NAN values with actual np.nan
    nan_value = -32768
    a[a == nan_value] = np.nan

    # making up the lon/lat grid. Might need adaptation to match other global data
    lons = np.linspace(0, 360, arr.shape[1])
    lats = np.linspace(70, -70, arr.shape[0])

    # create xarray DataArray from the dask array (thankfully supported, but not documented)
    img = xr.DataArray(a, dims=["lat", "lon"], coords={"lat": lats, "lon": lons})
    img.name = "H"
    return img

In [ ]:
hpar_path

PosixPath('/luna4/maye/h_parameter_maps/hpar_global128ppd_v1c.bin')

Size in bytes:

In [ ]:
size = hpar_path.stat().st_size
size

3303270404

Scaling down 80 lats to 70:

In [ ]:
n_lats = 20480 / 80 * 70
n_lats

17920.0

In [ ]:
npix = 46080 * int(n_lats)
npix

825753600

multiply by 32 bits, divide by 8 to get bytes:

In [ ]:
expected_bytes = npix * 32 / 8
expected_bytes

3303014400.0

Ratio:

In [ ]:
size / expected_bytes

1.0000775061713325

In [ ]:
npix = 46081 * (int(n_lats) + 1)
npix

825817601

In [ ]:
expected_bytes = npix * 32 / 8
size / expected_bytes

1.0

In [ ]:
lats = memmap_binary(lats_path)
lats[0], lats[-1]

(memmap([69.99609, 69.99609, 69.99609, ..., 69.99609, 69.99609, 69.99609],
        dtype=float32),
 memmap([-70.00391, -70.00391, -70.00391, ..., -70.00391, -70.00391,
         -70.00391], dtype=float32))

In [ ]:
lons = memmap_binary(lons_path)
lons[:, 0], lons[:, -1]

(memmap([0.003906, 0.003906, 0.003906, ..., 0.003906, 0.003906, 0.003906],
        dtype=float32),
 memmap([360.0039, 360.0039, 360.0039, ..., 360.0039, 360.0039, 360.0039],
        dtype=float32))

In [ ]:
with np.printoptions(precision=10):
    print(lons[0])

[3.9059999e-03 1.1719000e-02 1.9531000e-02 ... 3.5998828e+02 3.5999609e+02
 3.6000391e+02]


In [ ]:
with np.printoptions(precision=10):
    print(lons[:, 0])

[0.003906 0.003906 0.003906 ... 0.003906 0.003906 0.003906]


In [ ]:
np.printopt

In [ ]:
with np.printoptions(precision=20):
    print(lons[0])

[3.9059999e-03 1.1719000e-02 1.9531000e-02 ... 3.5998828e+02 3.5999609e+02
 3.6000391e+02]


In [ ]:
np.linspace(-70, 70, int(n_lats))

array([-70.        , -69.99218706, -69.98437413, ...,  69.98437413,
        69.99218706,  70.        ])

In [ ]:
half_pixel_degree = 1 / 128 / 2
half_pixel_degree

0.00390625

In [ ]:
with_n_plus_1 = np.linspace(
    -70 - half_pixel_degree, 70 - half_pixel_degree, int(n_lats) + 1
)

In [ ]:
from_data = lats[
    :,
    0,
][::-1]

In [ ]:
n = len(from_data)

In [ ]:
%matplotlib widget

In [ ]:
from_data[:10]

memmap([-70.00391, -69.99609, -69.98828, -69.98047, -69.97266, -69.96484,
        -69.95703, -69.94922, -69.94141, -69.93359], dtype=float32)

In [ ]:
with_n_plus_1

array([-70.00390625, -69.99609375, -69.98828125, ...,  69.98046875,
        69.98828125,  69.99609375])

In [ ]:
# export
class HReader:
    """Data reader class for H parameter map.

    It accesses preproduced data for 128 ppd for DEM, slope, and aspect,
    located on the luna4 disk.

    It uses virtual dask.Arrays so that virtually no memory is consumed until you
    resolve a chain of operations with the `.compute()` call.

    Attributes
    ----------
    H: xarray.DataArray
        H parameter map
    """

    def __init__(self, lat_limit=None, other_path=None):
        self.lat_limit = lat_limit
        path = hpar_path if other_path is None else other_path
        arr = memmap_binary(path)
        # dask array:
        a = da.from_array(arr)
        # replace NAN values with np.nan
        a[a == -32768] = np.nan

        # As per Paul's README:
        lats = np.linspace(70, -70, arr.shape[0], endpoint=True)
        lons = np.linspace(0, 360, arr.shape[1], endpoint=True)

        # create xarray DataArray from the dask array (thankfully supported, but not documented)
        img = xr.DataArray(a, dims=["lat", "lon"], coords={"lat": lats, "lon": lons})
        img.name = "H"
        img.attrs["long_name"] = "H Parameter"
        img.attrs["units"] = "H units :-P"
        self.img = img

    def slice_lat(self, lat):
        """Return the map `data` constrained to lat <= `lat`.

        Parameters
        ----------
        data: {'dem', 'slope','aspect'}
            String that choses which data product should be constrained.
        lat: int, float
            Limiting latitude value.
        """
        s = slice(lat, -lat)
        return self.img.sel(lat=s, drop=True)

    def convert_to_lon180(self):
        "Switch image to -180/180 longitude system."
        p = self.img
        p.coords["lon"] = ((p.lon + 180) % 360) - 180
        with dask.config.set(**{"array.slicing.split_large_chunks": False}):
            self.img = p.sortby(p.lon)

    def convert_to_lon360(self):
        "Switch image to 360 longitude system."
        p = self.img
        p.coords["lon"] = p.coords["lon"] % 360
        with dask.config.set(**{"array.slicing.split_large_chunks": False}):
            self.img = p.sortby(p.lon)

    def assign_new_latlon(self, lat, lon):
        "make sure longitude layout matches!"
        p = self.img
        p.coords["lon"] = lon
        p.coords["lat"] = lat
        self.img = p.sortby(p.lon)

    def get_H_by_pixel(self, ilat, ilon):
        "Note: Decided to not apply offset!"
        val = self.img.isel(lat=ilat, lon=ilon)
        return float(val)

    def get_H_by_coord(self, lat, lon):
        "Note: Decided to not apply offset!"
        val = self.img.sel(lat=lat, lon=lon, method="nearest")
        return float(val)

    def plot_dem(self, lat_min, lon_min, dlat=1, dlon=1, lat_max=None, lon_max=None):
        sliced = self.get_slice("dem", lat_min, lon_min, dlat, dlon, lat_max, lon_max)
        return sliced.hvplot(cmap="viridis", aspect="equal", title="DEM")

    def plot_slope(self, lat_min, lon_min, dlat=1, dlon=1, lat_max=None, lon_max=None):
        sliced = self.get_slice("slope", lat_min, lon_min, dlat, dlon, lat_max, lon_max)
        return sliced.hvplot(cmap="inferno", aspect="equal", title="Slope [degrees]")

    def plot_aspect(self, lat_min, lon_min, dlat=1, dlon=1, lat_max=None, lon_max=None):
        sliced = self.get_slice(
            "aspect", lat_min, lon_min, dlat, dlon, lat_max, lon_max
        )
        return sliced.hvplot(cmap="twilight", aspect="equal", title="Azimuth [degrees]")

    def get_slice(
        self, obj, lat_min, lon_min, dlat=None, dlon=None, lat_max=None, lon_max=None
    ):
        """Slice a rectangular data tile out of the map.

        Parameters
        ----------
        obj: {'dem','slope','aspect'}
            Determines the object that will be sliced
        lat_min: float
            Lower left corner latitude of slice.
        lon_min: float
            Lower left corner longitude of slice.
        dlat: float
            Delta to be added to `lat_min`
        dlon: float
            Delta lon to be added to `lon_min`
        lat_max: float
            Alternative for dlat, to set upper end of latitude interval.
        lon_max: float
            Alternative for dlon, to set right end of longitude interval.

        Returns
        -------
        xarray.DataArray
            Sliced for the provided coordinate values.
        """
        data = getattr(self, obj)
        if lon_max is None:
            lon_max = lon_min + dlon
        if lat_max is None:
            lat_max = lat_min + dlat
        return data.sel(lat=slice(lat_max, lat_min), lon=slice(lon_min, lon_max))

In [ ]:
H = HReader()

In [ ]:
H.img

<xarray.DataArray 'H' (lat: 17921, lon: 46081)>
dask.array<where, shape=(17921, 46081), dtype=float32, chunksize=(5792, 5792), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 70.0 69.99 69.98 69.98 ... -69.98 -69.98 -69.99 -70.0
  * lon      (lon) float64 0.0 0.007812 0.01562 0.02344 ... 360.0 360.0 360.0
Attributes:
    long_name:  H Parameter
    units:      H units :-P

In [ ]:
H.get_H_by_coord(20, 50)

0.07237300276756287

In [ ]:
H.img.sel(lat=slice(21, 20), lon=slice(120,121)).hvplot()

:Image   [lon,lat]   (H)

In [ ]:
H.convert_to_lon180()

In [ ]:
H.img

<xarray.DataArray 'H' (lat: 17921, lon: 46081)>
dask.array<getitem, shape=(17921, 46081), dtype=float32, chunksize=(5792, 5792), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 70.0 69.99 69.98 69.98 ... -69.98 -69.98 -69.99 -70.0
  * lon      (lon) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
Attributes:
    long_name:  H Parameter
    units:      H units :-P

In [ ]:
H.get_H_by_coord(20, 70)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
lons = memmap_binary(lons_path)

In [ ]:
lons.shape

(17921, 46081)

In [ ]:
pd.Series(lons[:, 0]).value_counts()

0.003906    17921
dtype: int64

In [ ]:
pd.Series(lons[:, -1]).value_counts()

360.003906    17921
dtype: int64

In [ ]:
H = memmap_binary(hpar_path)

In [ ]:
H.shape

(17921, 46081)

In [ ]:
H[:, 0][:10]

memmap([0.078986, 0.078389, 0.07891 , 0.078347, 0.07683 , 0.0854  ,
        0.077884, 0.081484, 0.079931, 0.075148], dtype=float32)

In [ ]:
H[:, -1][:10]

memmap([0.078389, 0.07891 , 0.078347, 0.07683 , 0.0854  , 0.077884,
        0.081484, 0.079931, 0.075148, 0.073657], dtype=float32)

In [ ]:
H = read_hpar_binary()

In [ ]:
H.isel(lon=0).hvplot() * H.isel(lon=-1).hvplot()

:Overlay
   .Curve.I  :Curve   [lat]   (H)
   .Curve.II :Curve   [lat]   (H)

In [ ]:
(H.isel(lon=0)[1:] - H.isel(lon=-1)[:-1]).compute().data

array([-0.000521,  0.000563,  0.001517, ...,  0.000944,       nan,
             nan], dtype=float32)

In [ ]:
left = H.isel(lon=-1)[:-1].compute().data

In [ ]:
right = H.isel(lon=0)[1:].compute().data

In [ ]:
pd.Series(left - right).value_counts()

 0.000000    17326
-0.000407        2
-0.000293        2
 0.000192        1
 0.000830        1
             ...  
-0.001465        1
-0.000870        1
-0.001488        1
-0.001567        1
 0.002393        1
Length: 142, dtype: int64

In [ ]:
17326 / left.shape[0]

0.9668526785714285

In [ ]:
%matplotlib widget

In [ ]:
fig, ax = plt.subplots()
ax.plot(H[:, 0], label="left")
ax.plot(H[:, -1], label="right")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
H[:, 0][:10]

memmap([0.078986, 0.078389, 0.07891 , 0.078347, 0.07683 , 0.0854  ,
        0.077884, 0.081484, 0.079931, 0.075148], dtype=float32)

In [ ]:
H[:, -1][:10]

memmap([0.078389, 0.07891 , 0.078347, 0.07683 , 0.0854  , 0.077884,
        0.081484, 0.079931, 0.075148, 0.073657], dtype=float32)

In [ ]:
H.lon.diff("lon") / 2

<xarray.DataArray 'lon' (lon: 46080)>
array([0.00390625, 0.00390625, 0.00390625, ..., 0.00390625, 0.00390625,
       0.00390625])
Coordinates:
  * lon      (lon) float64 0.007812 0.01562 0.02344 ... 360.0 360.0 360.0

In [ ]:
H = read_hpar_binary()

In [ ]:
import hvplot.xarray

In [ ]:
H.isel(lon=0).hvplot() * H.isel(lon=-1).hvplot()

:Overlay
   .Curve.I  :Curve   [lat]   (H)
   .Curve.II :Curve   [lat]   (H)

In [ ]:
rate = 47

In [ ]:
rate * 8

376

In [ ]:
arr_x = []
arr_y = []
for i in range(5):
    arr_x.append(np.sort(np.random.uniform(size=20)))
    arr_y.append(np.sort(np.random.uniform(size=20)))

In [ ]:
plt.figure()
for x, y in zip(arr_x, arr_y):
    plt.plot(x, y)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …